In [1]:
import numpy as np
import pandas as pd

In [58]:
schedule = pd.read_csv("C:/Users/njj06/Desktop/ESC_summer/Baseball_ChilliShrimp/data/schedule.csv", encoding = 'CP949')
pa_b = pd.read_csv("https://raw.githubusercontent.com/njj06135/Baseball_ChilliShrimp/master/data/batter_predict_pa.csv")
modeling_b = pd.read_csv("https://raw.githubusercontent.com/njj06135/Baseball_ChilliShrimp/master/data/batter_feature_selection.csv")

### batter

In [59]:
batter = modeling_b.loc[(modeling_b['year']==2020) & (modeling_b['month']>7)]

In [60]:
batter = batter[['T_ID', 'P_ID']].drop_duplicates()

In [61]:
pa_b['P_ID']=pa_b['P_ID'].astype('int')

In [62]:
batter_left= pd.merge(pa_b, batter, left_on='P_ID', right_on='P_ID', how='left')
batter_left

,year,month,P_ID,PA,T_ID
0,2020,10,65304,3.0,WO
1,2020,10,64012,10.0,KT
2,2020,10,68204,4.0,OB
3,2020,10,66203,6.0,OB
4,2020,10,50293,2.0,OB
5,2020,10,64213,8.0,OB
6,2020,10,62356,3.0,WO
7,2020,10,79192,22.0,LG
8,2020,10,64469,3.0,SS
9,2020,10,60343,28.0,HH


In [63]:
batter_2020 = pd.read_csv("C:/Users/njj06/Desktop/ESC_summer/Baseball_ChilliShrimp/data/batter_babip_2020.csv")

In [64]:
batter_tidy = pd.read_csv("C:/Users/njj06/Desktop/batter_tidy.csv")

In [65]:
babip_total = batter_tidy.groupby(['P_ID']).sum()

In [66]:
babip_total['BABIP'] = (babip_total['HIT'] - babip_total['HR']) / (babip_total['AB'] - babip_total['KK'] - babip_total['HR'] + babip_total['SF'])

In [67]:
babip_total = babip_total[['BABIP']]

In [114]:
babip_total = babip_total.fillna(0)

In [115]:
babip_total

,BABIP,P_ID
0,0.250000,50054
1,0.277778,50066
2,0.434783,50150
3,0.331950,50165
4,0.240000,50202
5,0.000000,50203
6,0.000000,50204
7,0.000000,50208
8,0.266667,50293
9,0.000000,50296


In [116]:
batter_2020 = batter_tidy.loc[batter_tidy.year == 2020, :]
batter_2020 = batter_2020.groupby(['P_ID']).sum()

In [117]:
batter_2020['BABIP_2020'] = (batter_2020['HIT'] - batter_2020['HR']) / (batter_2020['AB'] - batter_2020['KK'] - batter_2020['HR'] + batter_2020['SF'])

In [118]:
babip_2020 = batter_2020[['BABIP_2020']]

In [119]:
babip_total['P_ID'] = babip_total.index
babip_total = babip_total.reset_index(drop = True)

In [120]:
babip_2020['P_ID'] = babip_2020.index

C:\Users\njj06\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [121]:
babip_2020 = babip_2020.reset_index(drop = True)

In [122]:
batter_left= pd.merge(batter_left, babip_total, left_on='P_ID', right_on='P_ID', how='left')
batter_left= pd.merge(batter_left, babip_2020, left_on='P_ID', right_on='P_ID', how='left')


In [123]:
batter_left

,year,month,P_ID,T_ID,PA,pred_inplay_hit,H2_rate,H3_rate,HR_rate,BB_rate,HP_rate,KK_rate,BK_rate,BABIP,BABIP_2020
0,2020,10,65304,WO,3.0,1.500000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000
1,2020,10,64012,KT,10.0,1.636905,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.227273
2,2020,10,68204,OB,4.0,0.888889,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.315789
3,2020,10,66203,OB,6.0,1.156571,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.352941
4,2020,10,50293,OB,2.0,0.380952,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.266667
5,2020,10,64213,OB,8.0,2.090119,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.361111
6,2020,10,62356,WO,3.0,2.012201,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.263158
7,2020,10,79192,LG,22.0,5.624222,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.334677
8,2020,10,64469,SS,3.0,0.742601,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.259259
9,2020,10,60343,HH,28.0,5.002962,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.289855


In [124]:
batter_left['total_BABIP']=batter_left['BABIP_2020']-(batter_left['BABIP_2020']-batter_left['BABIP'])/5
batter_left

,year,month,P_ID,T_ID,PA,pred_inplay_hit,H2_rate,H3_rate,HR_rate,BB_rate,HP_rate,KK_rate,BK_rate,BABIP,BABIP_2020,total_BABIP
0,2020,10,65304,WO,3.0,1.500000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,NaN
1,2020,10,64012,KT,10.0,1.636905,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.227273,NaN
2,2020,10,68204,OB,4.0,0.888889,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.315789,NaN
3,2020,10,66203,OB,6.0,1.156571,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.352941,NaN
4,2020,10,50293,OB,2.0,0.380952,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.266667,NaN
5,2020,10,64213,OB,8.0,2.090119,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.361111,NaN
6,2020,10,62356,WO,3.0,2.012201,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.263158,NaN
7,2020,10,79192,LG,22.0,5.624222,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.334677,NaN
8,2020,10,64469,SS,3.0,0.742601,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.259259,NaN
9,2020,10,60343,HH,28.0,5.002962,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.289855,NaN


In [125]:
data = pd.read_csv("C:/Users/njj06/Desktop/batter_tidy.csv")

In [126]:
df1 = data.copy()
df1.head()

,year,month,P_ID,T_ID,AB,HIT,H2,H3,HR,BB,...,VS_T_ID_KT,VS_T_ID_LG,VS_T_ID_LT,VS_T_ID_NC,VS_T_ID_OB,VS_T_ID_SK,VS_T_ID_SS,VS_T_ID_WO,HOME,AWAY
0,2016,4,60146,LG,0,0,0,0,0,0,...,2,0,3,3,0,3,2,3,10,12
1,2016,4,60263,NC,0,0,0,0,0,0,...,0,3,4,0,3,3,3,2,9,14
2,2016,4,60404,HH,12,2,0,0,0,0,...,0,4,3,3,6,0,2,3,12,11
3,2016,4,60456,SS,65,15,3,1,0,5,...,6,2,3,3,4,0,0,0,10,13
4,2016,4,60523,LT,91,22,4,1,0,7,...,3,3,0,4,0,3,3,3,14,11


In [127]:
df1 = df1.loc[df1['year']==2020]

In [128]:
df1

,year,month,P_ID,T_ID,AB,HIT,H2,H3,HR,BB,...,VS_T_ID_KT,VS_T_ID_LG,VS_T_ID_LT,VS_T_ID_NC,VS_T_ID_OB,VS_T_ID_SK,VS_T_ID_SS,VS_T_ID_WO,HOME,AWAY
6273,2020,5,50066,KT,16,4,2,0,0,1,...,0,3,3,3,2,0,3,3,12,11
6274,2020,5,50150,LG,0,0,0,0,0,0,...,3,0,0,2,3,3,3,2,11,11
6275,2020,5,50165,LG,80,30,5,0,10,10,...,3,0,0,2,3,3,3,2,11,11
6276,2020,5,50202,OB,14,3,0,0,0,1,...,2,3,6,3,0,3,3,0,11,12
6277,2020,5,50300,WO,35,4,0,0,1,1,...,3,2,3,3,0,3,3,0,12,11
6278,2020,5,50458,SS,29,7,1,0,0,1,...,3,3,3,6,3,0,0,3,15,9
6279,2020,5,50468,SS,64,16,3,0,3,5,...,3,3,3,6,3,0,0,3,15,9
6280,2020,5,50506,LT,81,17,4,0,4,7,...,3,0,0,0,6,2,3,3,11,12
6281,2020,5,50854,SK,18,6,1,1,0,0,...,0,3,2,3,3,0,0,3,12,11
6282,2020,5,50923,NC,78,21,2,1,5,7,...,3,2,0,0,3,3,6,3,11,12


In [129]:
df1 = df1.groupby(['P_ID']).sum()
df1['inplay_ratio'] = (df1['AB']-df1['KK']-df1['HR']+df1['SF'])/df1['PA']

In [130]:
df1

,year,month,AB,HIT,H2,H3,HR,BB,HP,KK,...,VS_T_ID_LG,VS_T_ID_LT,VS_T_ID_NC,VS_T_ID_OB,VS_T_ID_SK,VS_T_ID_SS,VS_T_ID_WO,HOME,AWAY,inplay_ratio
P_ID,,,,,,,,,,,,,,,,,,,,,
50054,8080,30,65,13,3,0,0,7,1,13,...,8,9,12,10,12,10,10,45,47,0.712329
50066,6060,18,25,6,2,0,1,2,0,6,...,7,9,11,5,4,7,6,36,32,0.666667
50150,10100,35,29,10,2,0,0,1,0,6,...,0,12,11,15,14,13,14,56,61,0.766667
50165,10100,35,399,114,17,2,34,51,3,128,...,0,12,11,15,14,13,14,56,61,0.517167
50202,8080,26,28,6,0,0,0,1,0,4,...,13,14,14,0,10,12,6,47,46,0.833333
50203,2020,9,0,0,0,0,0,0,0,0,...,2,0,2,0,2,4,5,14,11,NaN
50204,6060,21,2,0,0,0,0,0,0,2,...,10,8,11,0,7,9,6,36,34,0.000000
50208,4040,14,4,0,0,0,0,0,0,1,...,4,6,10,0,4,9,1,21,26,0.750000
50293,4040,15,17,4,0,0,0,3,1,2,...,5,0,8,0,4,7,6,23,25,0.714286


In [131]:
df1['P_ID'] = df1.index

In [132]:
df1 = df1.reset_index(drop = True)

In [133]:
df1

,year,month,AB,HIT,H2,H3,HR,BB,HP,KK,...,VS_T_ID_LT,VS_T_ID_NC,VS_T_ID_OB,VS_T_ID_SK,VS_T_ID_SS,VS_T_ID_WO,HOME,AWAY,inplay_ratio,P_ID
0,8080,30,65,13,3,0,0,7,1,13,...,9,12,10,12,10,10,45,47,0.712329,50054
1,6060,18,25,6,2,0,1,2,0,6,...,9,11,5,4,7,6,36,32,0.666667,50066
2,10100,35,29,10,2,0,0,1,0,6,...,12,11,15,14,13,14,56,61,0.766667,50150
3,10100,35,399,114,17,2,34,51,3,128,...,12,11,15,14,13,14,56,61,0.517167,50165
4,8080,26,28,6,0,0,0,1,0,4,...,14,14,0,10,12,6,47,46,0.833333,50202
5,2020,9,0,0,0,0,0,0,0,0,...,0,2,0,2,4,5,14,11,NaN,50203
6,6060,21,2,0,0,0,0,0,0,2,...,8,11,0,7,9,6,36,34,0.000000,50204
7,4040,14,4,0,0,0,0,0,0,1,...,6,10,0,4,9,1,21,26,0.750000,50208
8,4040,15,17,4,0,0,0,3,1,2,...,0,8,0,4,7,6,23,25,0.714286,50293
9,2020,6,0,0,0,0,0,0,0,0,...,0,6,0,2,3,1,9,14,NaN,50296


In [134]:
df1.isna().sum()

year             0
month            0
AB               0
HIT              0
H2               0
H3               0
HR               0
BB               0
HP               0
KK               0
GD               0
PA               0
SB               0
CS               0
SF               0
BAT_ORDER        0
stadium_MH       0
stadium_MS       0
stadium_JS       0
stadium_SW       0
stadium_DJ       0
stadium_DG       0
stadium_GJ       0
stadium_GC       0
stadium_SJ       0
stadium_PH       0
stadium_US       0
stadium_CJ       0
stadium_CW       0
VS_T_ID_HH       0
VS_T_ID_HT       0
VS_T_ID_KT       0
VS_T_ID_LG       0
VS_T_ID_LT       0
VS_T_ID_NC       0
VS_T_ID_OB       0
VS_T_ID_SK       0
VS_T_ID_SS       0
VS_T_ID_WO       0
HOME             0
AWAY             0
inplay_ratio    77
P_ID             0
dtype: int64

In [135]:
df1['P_ID']=df1['P_ID'].astype('int')

In [136]:
batter_left

,year,month,P_ID,T_ID,PA,pred_inplay_hit,H2_rate,H3_rate,HR_rate,BB_rate,HP_rate,KK_rate,BK_rate,BABIP,BABIP_2020,total_BABIP
0,2020,10,65304,WO,3.0,1.500000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,NaN
1,2020,10,64012,KT,10.0,1.636905,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.227273,NaN
2,2020,10,68204,OB,4.0,0.888889,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.315789,NaN
3,2020,10,66203,OB,6.0,1.156571,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.352941,NaN
4,2020,10,50293,OB,2.0,0.380952,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.266667,NaN
5,2020,10,64213,OB,8.0,2.090119,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.361111,NaN
6,2020,10,62356,WO,3.0,2.012201,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.263158,NaN
7,2020,10,79192,LG,22.0,5.624222,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.334677,NaN
8,2020,10,64469,SS,3.0,0.742601,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.259259,NaN
9,2020,10,60343,HH,28.0,5.002962,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.289855,NaN


In [137]:
data = data.loc[(data['year']==2020) & (data['month']==9)]

In [138]:
data

,year,month,P_ID,T_ID,AB,HIT,H2,H3,HR,BB,...,VS_T_ID_KT,VS_T_ID_LG,VS_T_ID_LT,VS_T_ID_NC,VS_T_ID_OB,VS_T_ID_SK,VS_T_ID_SS,VS_T_ID_WO,HOME,AWAY
7197,2020,9,65304,WO,2,2,0,0,0,0,...,2,2,2,2,5,4,2,0,14,13
7198,2020,9,64012,KT,7,4,1,0,1,2,...,0,2,3,2,4,4,4,2,13,12
7199,2020,9,68204,OB,4,2,1,0,0,0,...,4,2,0,2,0,2,4,5,14,11
7200,2020,9,66203,OB,6,3,1,0,0,0,...,4,2,0,2,0,2,4,5,14,11
7201,2020,9,50293,OB,2,1,0,0,0,0,...,4,2,0,2,0,2,4,5,14,11
7202,2020,9,64213,OB,6,3,0,0,0,2,...,4,2,0,2,0,2,4,5,14,11
7203,2020,9,62356,WO,2,1,0,0,0,0,...,2,2,2,2,5,4,2,0,14,13
7204,2020,9,79192,LG,12,6,0,0,1,0,...,2,0,6,4,2,3,2,2,12,12
7205,2020,9,64469,SS,2,1,0,0,0,0,...,4,2,2,3,4,2,0,2,13,11
7206,2020,9,60343,HH,25,12,3,0,1,1,...,2,2,2,2,5,2,2,4,16,9


In [139]:
p_id = data['P_ID'].reset_index(drop = True)

In [140]:
batter_2020 = pd.DataFrame()

for i in range(len(p_id)):
    pid = p_id[i]
    batter = df1.loc[df1.P_ID == pid,:]
    batter_2020 = batter_2020.append(batter)
    

In [141]:
batter_2020 = batter_2020.reset_index(drop = True)

In [142]:
batter_2020

,year,month,AB,HIT,H2,H3,HR,BB,HP,KK,...,VS_T_ID_LT,VS_T_ID_NC,VS_T_ID_OB,VS_T_ID_SK,VS_T_ID_SS,VS_T_ID_WO,HOME,AWAY,inplay_ratio,P_ID
0,4040,17,2,2,0,0,0,0,0,0,...,7,4,5,4,6,0,29,24,0.666667,65304
1,10100,35,33,6,1,0,1,4,0,12,...,12,15,12,12,13,13,57,58,0.550000,64012
2,8080,30,47,12,1,0,0,3,0,13,...,8,13,0,9,13,11,50,45,0.703704,68204
3,6060,23,22,6,2,0,0,2,0,6,...,6,12,0,6,13,6,35,37,0.680000,66203
4,4040,15,17,4,0,0,0,3,1,2,...,0,8,0,4,7,6,23,25,0.714286,50293
5,8080,30,43,13,0,1,0,2,0,8,...,8,13,0,9,13,11,50,45,0.782609,64213
6,10100,35,96,21,2,1,1,5,1,23,...,16,13,11,13,16,0,65,59,0.710280,62356
7,10100,35,314,93,13,1,10,23,7,60,...,12,11,15,14,13,14,56,61,0.694678,79192
8,6060,22,31,7,4,0,0,2,1,4,...,8,4,7,8,0,9,36,34,0.771429,64469
9,10100,35,88,23,3,0,3,7,3,19,...,13,15,10,16,11,13,59,59,0.676471,60343


In [143]:
batter_left= pd.merge(batter_left, df1[['P_ID','inplay_ratio']], left_on='P_ID', right_on='P_ID', how='left')

In [144]:
batter_left.index = batter_left['P_ID']
df1 = df1.reset_index(drop = True)

In [145]:
batter_left = batter_left.reset_index(drop = True)

In [146]:
batter_2020['inplay'] = batter_2020['AB'] - batter_2020['KK'] - batter_2020['HR'] + batter_2020['SF']

In [147]:
batter_2020['inplay_hit'] = batter_2020['HIT'] - batter_2020['HR']

In [148]:
batter_left= pd.merge(batter_left, batter_2020[['P_ID', 'inplay', 'inplay_hit']], left_on='P_ID', right_on='P_ID', how='left')

In [149]:
batter_left

,year,month,P_ID,T_ID,PA,pred_inplay_hit,H2_rate,H3_rate,HR_rate,BB_rate,HP_rate,KK_rate,BK_rate,BABIP,BABIP_2020,total_BABIP,inplay_ratio,inplay,inplay_hit
0,2020,10,65304,WO,3.0,1.500000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,NaN,0.666667,2.0,2
1,2020,10,64012,KT,10.0,1.636905,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.227273,NaN,0.550000,22.0,5
2,2020,10,68204,OB,4.0,0.888889,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.315789,NaN,0.703704,38.0,12
3,2020,10,66203,OB,6.0,1.156571,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.352941,NaN,0.680000,17.0,6
4,2020,10,50293,OB,2.0,0.380952,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.266667,NaN,0.714286,15.0,4
5,2020,10,64213,OB,8.0,2.090119,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.361111,NaN,0.782609,36.0,13
6,2020,10,62356,WO,3.0,2.012201,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.263158,NaN,0.710280,76.0,20
7,2020,10,79192,LG,22.0,5.624222,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.334677,NaN,0.694678,248.0,83
8,2020,10,64469,SS,3.0,0.742601,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.259259,NaN,0.771429,27.0,7
9,2020,10,60343,HH,28.0,5.002962,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.289855,NaN,0.676471,69.0,20


In [150]:
batter_left['pred_inplay'] = batter_left['PA'] * batter_left['inplay_ratio']

In [151]:
batter_left

,year,month,P_ID,T_ID,PA,pred_inplay_hit,H2_rate,H3_rate,HR_rate,BB_rate,HP_rate,KK_rate,BK_rate,BABIP,BABIP_2020,total_BABIP,inplay_ratio,inplay,inplay_hit,pred_inplay
0,2020,10,65304,WO,3.0,1.500000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,NaN,0.666667,2.0,2,2.000000
1,2020,10,64012,KT,10.0,1.636905,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.227273,NaN,0.550000,22.0,5,5.500000
2,2020,10,68204,OB,4.0,0.888889,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.315789,NaN,0.703704,38.0,12,2.814815
3,2020,10,66203,OB,6.0,1.156571,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.352941,NaN,0.680000,17.0,6,4.080000
4,2020,10,50293,OB,2.0,0.380952,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.266667,NaN,0.714286,15.0,4,1.428571
5,2020,10,64213,OB,8.0,2.090119,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.361111,NaN,0.782609,36.0,13,6.260870
6,2020,10,62356,WO,3.0,2.012201,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.263158,NaN,0.710280,76.0,20,2.130841
7,2020,10,79192,LG,22.0,5.624222,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.334677,NaN,0.694678,248.0,83,15.282913
8,2020,10,64469,SS,3.0,0.742601,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.259259,NaN,0.771429,27.0,7,2.314286
9,2020,10,60343,HH,28.0,5.002962,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.289855,NaN,0.676471,69.0,20,18.941176


In [152]:
batter_left['pred_inplay_hit'] = batter_left['total_BABIP'] * (batter_left['inplay'] + batter_left['pred_inplay']) - batter_left['inplay_hit']

In [153]:
batter_left = batter_left[['year', 'month', 'P_ID', 'T_ID', 'PA', 'pred_inplay_hit']]

In [154]:
batter_left = batter_left.fillna(0)

In [155]:
batter_left.pred_inplay_hit[batter_left.pred_inplay_hit < 0.0 ]=0

C:\Users\njj06\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [156]:
batter_left

,year,month,P_ID,T_ID,PA,pred_inplay_hit
0,2020,10,65304,WO,3.0,0.0
1,2020,10,64012,KT,10.0,0.0
2,2020,10,68204,OB,4.0,0.0
3,2020,10,66203,OB,6.0,0.0
4,2020,10,50293,OB,2.0,0.0
5,2020,10,64213,OB,8.0,0.0
6,2020,10,62356,WO,3.0,0.0
7,2020,10,79192,LG,22.0,0.0
8,2020,10,64469,SS,3.0,0.0
9,2020,10,60343,HH,28.0,0.0


In [157]:
batter_2020_ratio = pd.read_csv("C:/Users/njj06/Desktop/ESC_summer/Baseball_ChilliShrimp/data/batter_2020_ratio.csv")

In [158]:
batter_2020_ratio

,P_ID,H2_rate,H3_rate,HR_rate,BB_rate,HP_rate,KK_rate,BK_rate
0,50030,0.166667,0.000000,0.055556,0.075107,0.010730,0.141631,0.000000
1,50036,0.111111,0.111111,0.111111,0.151515,0.090909,0.090909,0.000000
2,50040,0.141243,0.005650,0.079096,0.073713,0.009736,0.173853,0.000000
3,50109,0.166667,0.000000,0.000000,0.050000,0.050000,0.050000,0.000000
4,50126,0.219178,0.000000,0.054795,0.084795,0.026316,0.143275,0.002924
5,50157,0.172414,0.000000,0.103448,0.076233,0.026906,0.147982,0.000000
6,50203,0.083333,0.000000,0.083333,0.048780,0.048780,0.219512,0.000000
7,50234,0.152778,0.000000,0.069444,0.081818,0.003030,0.251515,0.000000
8,50262,0.333333,0.000000,0.000000,0.368421,0.000000,0.105263,0.000000
9,50296,0.166667,0.000000,0.166667,0.225806,0.032258,0.193548,0.000000


In [159]:
batter_left = pd.merge(batter_left, batter_2020_ratio, left_on='P_ID', right_on='P_ID', how='left')

In [160]:
batter_left

,year,month,P_ID,T_ID,PA,pred_inplay_hit,H2_rate,H3_rate,HR_rate,BB_rate,HP_rate,KK_rate,BK_rate
0,2020,10,65304,WO,3.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020,10,64012,KT,10.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020,10,68204,OB,4.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020,10,66203,OB,6.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020,10,50293,OB,2.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2020,10,64213,OB,8.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2020,10,62356,WO,3.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,2020,10,79192,LG,22.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2020,10,64469,SS,3.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,2020,10,60343,HH,28.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
